In [3]:
import pandas as pd
import math

In [1]:
def _isLetter(letter):
    # 0 a 47
    # 58 a 63
    # 91 a 96
    # 123 a 126
    # 8200,8230
    
    codeLetter = ord(letter)

    return not ( 0 <= codeLetter <= 47 
    or 58 <= codeLetter <= 63 
    or 91 <= codeLetter <= 96 
    or 123 <= codeLetter <= 126 
    or 8200 <= codeLetter <= 8230)


In [5]:
chr(98)

'b'

In [5]:
def cleaningWord(word):

    i = 0
    n = len(word)
    
    while i < n and not _isLetter(word[i]):
        i += 1

    j = n
    while j > 0 and not _isLetter(word[j-1]):
        j -= 1
    
 
    word = word[i:j]
    
    return word.lower()

In [6]:
def tf(palavra, doc):
    # palavra : total de palavras que aparceu no doc
    # doc : numero total de palavras do doc
    return palavra/doc

In [7]:
def n_containing(palavra, docs):
    # retorna o numero de documento que tem a palvra
    pass

In [8]:
def idf(qntWords, docs):
    # Palavra: total de documentos que tem a palavra
    # docs: total de documentos
    h= docs/(qntWords if qntWords > 0 else 1)
    return math.log2(h)
  

In [9]:
def tfidf(palavra, doc,qntWords, docs):
    return tf(palavra, doc) * idf(qntWords, docs)

In [13]:
def loadStopWords():
    file_ = open("../data/stopwords_br.txt")
    wordsStop = []
    linhas = file_.readlines()

    for line in linhas:
        wordsStop.append(cleaningWord(line))

    return wordsStop

In [14]:
def validate(word,stopW):
    return not cleaningWord(word) in stopW


In [15]:
def getTotWords(file,stopWords,doc, start, end):
    wordsInDoc = {}
    totWord = 0

    for idxLine in range(start, end+1):
        
        wos = [ cleaningWord(x) for x in file[idxLine].split(" ") if validate(x,stopWords)]

        for wo in wos:
            totWord += 1
            if not wo in wordsInDoc.keys():
                wordsInDoc[wo] = 1
            else:
                wordsInDoc[wo] += 1
    return wordsInDoc, totWord

In [16]:
def mappingPages(file):
    totLines = len(file)

    pages = {}
    startPage = 0
    endPage = 0
    ultPage = 1
    i = 1

    while i < totLines:

        subStr = file[i][:2]

        if (subStr == "PA"):
            endPage = i-1
            pages[ultPage] = {
                "start":startPage,
                "end": endPage
            }

            startPage = i
            ultPage += 1

        i+=1


    endPage = i-1
    pages[ultPage] = {
        "start":startPage,
        "end": endPage
    }

    return pages

    

In [17]:
def preprareTable(dicResult, totDocs, totWords):
    
    words = []
    recurrences = []
    pontuation = []
    pagesOc = []

    for wd in dicResult:
        words.append(wd)
        recurrences.append(dicResult[wd]["totRecorences"])

        pont = 0
        stringTmp = ""
        for page in dicResult[wd]["pages"]:
            tmp = tfidf(dicResult[wd]["pages"][page]["recorrences"], dicResult[wd]["pages"][page]["totWord"], dicResult[wd]["totPages"], totDocs)
            pont += tmp
            stringTmp = f'{stringTmp} -> {page}: {dicResult[wd]["pages"][page]["totWord"]}'

        toP = dicResult[wd]["totPages"] if dicResult[wd]["totPages"] > 0 else 1

        pont = pont/toP

        pontuation.append( pont )
        pagesOc.append( stringTmp )

    
    d = {'palavra': words, 'rec': recurrences, "pont": pontuation}
        
    return pd.DataFrame(data=d)


In [30]:
# file_ = open("../data/Paralelismo.base")
# ffile_ = open("../data/Guarani.base")
file_ = open("../data/Aventuras.base")
lines = file_.readlines()

pages = mappingPages(lines)
print(f"Pages: {len(pages)}")

Pages: 264


In [28]:
words = {}
wordsResul = {}

wordCont = 0

stopWords = loadStopWords()

for page in pages:

    start = pages[page]["start"]
    end = pages[page]["end"]

    words, totWord = getTotWords(lines,stopWords, page, start, end)
    for wd in words:
        if len(wd) == 0:
            continue

        wordCont += 1
            
        if not wd in wordsResul.keys():
            wordsResul[wd] = {
                "pages":{
                    page:{
                        "recorrences": words[wd],
                        "totWord": totWord
                    }
                },
                "totPages": 1,
                "totRecorences": words[wd]
            }
        else:
            wordsResul[wd]["pages"][page] = {
                "recorrences": words[wd],
                "totWord": totWord
            }
            wordsResul[wd]["totPages"] += 1
            wordsResul[wd]["totRecorences"] += words[wd]

In [26]:
table = preprareTable(wordsResul, len(pages), wordCont)

# table = table.loc[table["rec"] > 1]
# table = table.loc[table["rec"] <= 5]
table = table.sort_values(by="pont", ascending=False)
print(table)

table.to_csv("Paralelismo.csv")


        palavra  rec      pont
8865  myrtaceae    6  0.206216
8746     textur    5  0.203480
4780      ducto    9  0.179122
8742         ní    4  0.162784
8743     zvaeis    4  0.162784
...         ...  ...       ...
1331      sendo  133  0.004849
255   apresenta  174  0.004406
1306       modo  272  0.004107
397    capítulo  277  0.002507
0            pa  295  0.000000

[9747 rows x 3 columns]
